# Import Libraries

In [53]:
import gurobipy as grb
import pandas as pd
from gurobipy import GRB
import numpy as np

# Model

In [54]:
 # Create a model
model = grb.Model("Model")

# Get data from json

In [55]:
# get the  data from the json file small.json
import json
with open('medium.json') as f:
    data = json.load(f)
# data

## Constants from data

In [56]:
horizon= data['horizon']
qualifications=data['qualifications']
staff_names=[i['name'] for i in data['staff']]
staff_qualifications=[i['qualifications'] for i in data['staff']]
job_list=[i['name'] for i in data['jobs']]
jour_list=[i for i in range(1,horizon+1)]
job_penality=[i['daily_penalty'] for i in data['jobs']] 
job_qualifications= [i['working_days_per_qualification'] for i in data['jobs']]
job_gain=[i['gain'] for i in data['jobs']]
due_dates=[i['due_date'] for i in data['jobs']]
vacation_staff=[i['vacations'] for i in data['staff']]

In [57]:
data

{'horizon': 22,
 'qualifications': ['F', 'G', 'A', 'D', 'J', 'C', 'H', 'E', 'B', 'I'],
 'staff': [{'name': 'Olivia',
   'qualifications': ['A', 'B', 'C'],
   'vacations': [1, 2]},
  {'name': 'Liam', 'qualifications': ['A', 'D', 'E'], 'vacations': [1, 2]},
  {'name': 'Emma', 'qualifications': ['B', 'H'], 'vacations': [8, 9]},
  {'name': 'Noah',
   'qualifications': ['G', 'D', 'J', 'C', 'H', 'I'],
   'vacations': []},
  {'name': 'Amelia',
   'qualifications': ['F', 'G', 'J', 'E'],
   'vacations': [16, 15]}],
 'jobs': [{'name': 'Job1',
   'gain': 15,
   'due_date': 20,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'B': 4}},
  {'name': 'Job2',
   'gain': 30,
   'due_date': 16,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'B': 2, 'C': 1, 'D': 4}},
  {'name': 'Job3',
   'gain': 30,
   'due_date': 12,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'C': 1}},
  {'name': 'Job4',
   'gain': 30,
   'due_date': 18,
   'daily_penal

# Model's parameters

#### Objective 1
H : days

Q : qualifications

S : staff

J : jobs

qi : qualifications of i

qj : qualifications of j

vi : vacation days of i

njk : required work load for j with k

cj : daily penalty of j

gj : gain of j

dj : due date of j

yj : is j completely done

lj : delay in days for j

ej : end date of j

pi_j_k_t : work is done by i with k for j on t

#### Objective 2
aij : j is assigned to i

ni : number of jobs i works on

max_i : max of ni

### Objective 3
sj : start date for j

nj : length in days for j

max_j : maximum of nj

In [58]:
H = [i for i in range(1,horizon+1)]
Q = qualifications
S = staff_names
J = job_list
# Parameters for each staff

qi = {}
for i in range(len(staff_names)) :
    k = staff_names[i]
    qi[k] = []
    for c in qualifications:
        if c in data['staff'][i]['qualifications']:
            qi[k].append(c)
vi={}
for ind,i in enumerate(staff_names):
    vi[i] = []
    for j in jour_list:
        if j in vacation_staff[ind]:
            vi[i].append(j)
#vi=staff_in_vacation.to_dict(orient='index')

# Parameters for each job
qj = dict()
for ind, j in enumerate(job_list):
    qj[j] = list(job_qualifications[ind].keys())
njk = dict(zip(job_list,job_qualifications))
cj=job_penality_dict=dict(zip(job_list,job_penality))
gj=job_gain_dict=dict(zip(job_list,job_gain))
dj = dict(zip(job_list,due_dates))


# Decision variables

In [59]:
# Objective 1
pijkt = model.addVars(staff_names,job_list,qualifications,jour_list, vtype=GRB.BINARY, name="pijkt")

yj = model.addVars(job_list, vtype=GRB.BINARY, name="yj")
lj = model.addVars(job_list, vtype=GRB.INTEGER, name="lj")
ej = model.addVars(job_list, vtype=GRB.INTEGER, name="ej")

# Objective 2
max_i = model.addVar(vtype=GRB.INTEGER, name="max_i")
ni = model.addVars(S, vtype=GRB.INTEGER, name="ni")
aij = model.addVars(S, J, vtype=GRB.BINARY, name="aij")

# Objective 3
sj = model.addVars(J, vtype=GRB.INTEGER, name="sj")
nj = model.addVars(J, vtype=GRB.INTEGER, name="nj")
max_j = model.addVar(vtype=GRB.INTEGER, name="max_j")

# Constraints

### Objective 1

In [60]:
constr1 = {f'constr1{i}_{t}':model.addConstr( grb.quicksum(pijkt[i,j,k,t] for j in J for k in Q) <= 1 , name=f"constr1{i}_{t}")
  for i in S
  for t in H}

constr2 = {f'constr2{i}_{t}':model.addConstr( grb.quicksum(pijkt[i,j,k,t] for j in J for k in Q) == 0 , name=f"constr2{i}_{t}")
  for i in S
  for t in vi[i]}

constr3 = {f'constr3{i}_{j}_{k}_{t}':model.addConstr( pijkt[i,j,k,t] == 0 , name=f"constr3{i}_{j}_{k}_{t}")
  for i in S
  for j in J
  for k in Q if k not in qj[j] or k not in qi[i]
  for t in H}

constr4 = {f'constr4{j}_{k}': model.addConstr( yj[j]*njk[j][k] <= grb.quicksum(pijkt[i,j,k,t] for i in S for t in H) , name=f"constr4{j}_{k}")
  for j in J
  for k in qj[j]}

constr5 = {f'constr5{j}_{k}': model.addConstr( grb.quicksum(pijkt[i,j,k,t] for i in S for t in H) <= njk[j][k] , name=f"constr5{j}_{k}")
  for j in J
  for k in qj[j]}

constr6 = {f'constr6{i}_{j}_{k}_{t}':model.addConstr( pijkt[i,j,k,t]*t <= ej[j] , name=f"constr6{i}_{j}_{k}_{t}")
  for i in S
  for j in J
  for k in Q
  for t in H}

constr7 = {f'constr7{j}' : model.addConstr( ej[j] - dj[j] <= lj[j] , name =f"constr7{j}")
  for j in J}

constr8 = {f'constr8{j}' : model.addConstr( 1 <= ej[j] , name =f"constr8{j}")
  for j in J}

constr9 = {f'constr9{j}' : model.addConstr( ej[j] <= H[-1] , name =f"constr9{j}")
  for j in J}

### Objective 2

In [61]:
# assigned jobs to a staff member i is aij[i, j]
constr10 = {f'constr10{i}_{j}_{k}_{t}' : model.addConstr( pijkt[i,j,k,t] <= aij[i, j] , name = f"constr10{i}_{j}_{k}_{t}")
    for i in S
    for j in J
    for k in Q
    for t in H}

# number of jobs assigned to a staff member i is ni[i]
constr11 = {f'constr11{i}' : model.addConstr( grb.quicksum(aij[i, j] for j in J) <= ni[i] , name = f"constr11{i}")
    for i in S}

# max_i max number of jobs assigned to a staff member
# for all staff i, number of jobs assigned to i is less than or equal to max_i
constr12 = {f'constr12{i}' : model.addConstr( ni[i] <= max_i , name = f"constr12{i}")
    for i in S}

### Objective 3

In [62]:
# start date is at least 1
constr14 = {f'constr14_{j}' : model.addConstr( 1 <= sj[j] , name = f"constr14_{j}")
    for j in J}

# start date of j is sj[j]
constr15 = {f'constr15{i}_{j}_{k}_{t}' : model.addConstr( sj[j] <= t*pijkt[i,j,k,t] + H[-1]*(1-pijkt[i,j,k,t]) , name = f"constr15{i}_{j}_{k}_{t}")
    for i in S
    for j in J
    for k in Q
    for t in H}

# length of a job j is nj[j]
constr16 = {f'constr16{j}' : model.addConstr( ej[j] + 1 - sj[j] <= nj[j] , name = f"constr16{j}")
    for j in J}

# max_j max of nj[j]
constr17 = {f'constr17{j}' : model.addConstr( nj[j] <= max_j , name = f"constr17{j}")
    for j in J}

# Objectives

In [98]:
# Objective 1
# model.setObjective( grb.quicksum((gj[j]*yj[j] - lj[j]*cj[j]) for j in J) , GRB.MAXIMIZE)

# Objective 1 & 2
# model.setObjective( grb.quicksum((gj[j]*yj[j] - lj[j]*cj[j]) for j in J) - max_i , GRB.MAXIMIZE)

# Objective 1 & 2 & 3
# model.setObjective( grb.quicksum((gj[j]*yj[j] - lj[j]*cj[j]) for j in J) - max_i - max_j , GRB.MAXIMIZE)

# setObjectiveN
model.ModelSense = GRB.MAXIMIZE
model.setObjectiveN( grb.quicksum((gj[j]*yj[j] - lj[j]*cj[j]) for j in J) , 0, 1)
model.setObjectiveN( - max_i , 1, 1)
model.setObjectiveN( - max_j , 2, 1)

# Paramétrage (mode mute)
model.params.outputflag = 1
# Résolution du PL
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 64900 rows, 16657 columns and 138818 nonzeros
Model fingerprint: 0xc7362fe2
Variable types: 0 continuous, 16657 integer (16590 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 64900 rows, 16657 columns and 138818 nonzeros
Model fingerpr

  Inf proof: 7
  Zero half: 18
  RLT: 33
  Relax-and-lift: 494

Explored 24909 nodes (8050574 simplex iterations) in 111.26 seconds (243.82 work units)
Thread count was 8 (of 8 available processors)

Solution count 10: 399 398 397 ... 332

Optimal solution found (tolerance 1.00e-04)
Best objective 3.990000000000e+02, best bound 3.990000000000e+02, gap 0.0000%

---------------------------------------------------------------------------
Multi-objectives: solved in 111.31 seconds (243.82 work units), solution count 10



In [99]:
#model.computeIIS()
model.write("model.lp")

In [100]:
# Query number of multiple objectives, and number of solutions
nSolutions  = model.SolCount
nObjectives = model.NumObj
print('Problem has', nObjectives, 'objectives')
print('Gurobi found', nSolutions, 'solutions')

Problem has 3 objectives
Gurobi found 10 solutions


In [101]:
solutions = []
for s in range(nSolutions):
  # Set which solution we will query from now on
    model.params.SolutionNumber = s

  # Print objective value of this solution in each objective
    print('Solution', s, ':', end='')
    for o in range(nObjectives):
        # Set which objective we will query
        model.params.ObjNumber = o
        # Query the o-th objective value
        print(' ',model.ObjNVal, end='')
    print('\n')

Solution 0 :  413.0  -6.0  -8.0

Solution 1 :  411.0  -6.0  -7.0

Solution 2 :  411.0  -6.0  -8.0

Solution 3 :  411.0  -6.0  -9.0

Solution 4 :  409.0  -6.0  -8.0

Solution 5 :  407.0  -5.0  -8.0

Solution 6 :  409.0  -6.0  -11.0

Solution 7 :  406.0  -6.0  -11.0

Solution 8 :  387.0  -6.0  -12.0

Solution 9 :  347.0  -5.0  -10.0



In [67]:
# Choix de la solution et de la fonction objective
model.params.SolutionNumber = 0
model.params.ObjNumber = 0
model.ObjNVal

413.0

In [68]:

values= []
for k, v in pijkt.items():
    values.append(v.Xn)

In [69]:
res = pd.DataFrame(list(pijkt.keys()))
res[4] = values
result = res[res[4] == 1]
result

,0,1,2,3,4
61,Olivia,Job1,A,18,1.0
62,Olivia,Job1,A,19,1.0
63,Olivia,Job1,A,20,1.0
64,Olivia,Job1,A,21,1.0
489,Olivia,Job3,A,6,1.0
...,...,...,...,...,...
15941,Amelia,Job13,J,14,1.0
16082,Amelia,Job14,G,1,1.0
16087,Amelia,Job14,G,6,1.0
16149,Amelia,Job14,J,2,1.0


In [97]:
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors


colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)

# Sort colors by hue, saturation, value and name.
by_hsv = sorted((tuple(mcolors.rgb_to_hsv(mcolors.to_rgba(color)[:3])), name)
                for name, color in colors.items())
sorted_names = [name for hsv, name in by_hsv]

In [77]:
color_list = np.random.choice(sorted_names,15)
color_dict = dict(zip(job_list,color_list))
color_dict

{'Job1': 'bisque',
 'Job2': 'firebrick',
 'Job3': 'lavender',
 'Job4': 'lightpink',
 'Job5': 'mediumblue',
 'Job6': 'lightcoral',
 'Job7': 'lightslategray',
 'Job8': 'gainsboro',
 'Job9': 'indianred',
 'Job10': 'rebeccapurple',
 'Job11': 'lavender',
 'Job12': 'palevioletred',
 'Job13': 'fuchsia',
 'Job14': 'ivory',
 'Job15': 'crimson'}

## Result Table

In [94]:
def color_table(x):
    
    if pd.isna(x):
        return "background-color: white"
    
    else:
        x = x.split(" ")[0]
        if x in color_dict.keys():
            color = color_dict[x]
            return "background-color: " + color
        else:
           return  "background-color: red"

In [95]:
df = pd.DataFrame(columns = [i for i in range(1,23)], index = staff_names)

for ind, val in result.iterrows():
    col = val[3]
    row = val[0]
    v = val[1] + " " + val[2]
    df.loc[row,col] = v
    # vacation
    for staff in staff_names:
        for day in jour_list:
            if day in vi[staff]:
                df.loc[staff,day] = 'X'
df.style.applymap(color_table)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
Olivia,X,X,Job5 C,Job5 C,Job5 C,Job3 A,Job5 C,Job3 A,Job3 A,Job4 A,Job3 C,Job3 A,Job4 A,Job4 A,Job4 A,Job4 A,Job4 A,Job1 A,Job1 A,Job1 A,Job1 A,nan
Liam,X,X,Job5 D,Job5 D,Job5 D,Job5 D,Job5 D,Job5 D,Job6 E,Job6 E,Job6 D,Job6 D,Job6 E,Job6 D,Job6 D,Job6 E,Job9 E,Job9 E,Job7 D,Job7 E,Job7 D,Job7 E
Emma,Job5 B,Job5 B,Job5 B,Job12 H,Job12 H,Job12 H,Job12 H,X,X,Job12 H,Job12 H,Job15 H,Job15 H,Job15 H,Job15 H,Job9 H,Job9 H,Job1 B,Job1 B,Job1 B,Job1 B,nan
Noah,Job14 G,Job14 G,Job14 J,Job5 D,Job5 D,Job14 J,Job13 I,Job13 I,Job15 G,Job13 I,Job13 J,Job13 I,Job13 J,Job15 G,Job15 G,Job15 G,Job9 G,Job9 G,Job9 G,Job7 D,Job7 D,Job7 D
Amelia,Job14 G,Job14 J,Job14 J,Job12 J,Job12 J,Job14 G,Job13 J,Job13 J,Job6 E,Job12 J,Job13 J,Job13 J,Job13 J,Job13 J,X,X,Job9 E,Job9 E,Job7 E,Job7 E,Job7 F,Job7 E


In [96]:
yj

{'Job1': <gurobi.Var yj[Job1] (value 1.0)>,
 'Job2': <gurobi.Var yj[Job2] (value -0.0)>,
 'Job3': <gurobi.Var yj[Job3] (value 1.0)>,
 'Job4': <gurobi.Var yj[Job4] (value 1.0)>,
 'Job5': <gurobi.Var yj[Job5] (value 1.0)>,
 'Job6': <gurobi.Var yj[Job6] (value 1.0)>,
 'Job7': <gurobi.Var yj[Job7] (value 1.0)>,
 'Job8': <gurobi.Var yj[Job8] (value -0.0)>,
 'Job9': <gurobi.Var yj[Job9] (value 1.0)>,
 'Job10': <gurobi.Var yj[Job10] (value 0.0)>,
 'Job11': <gurobi.Var yj[Job11] (value -0.0)>,
 'Job12': <gurobi.Var yj[Job12] (value 1.0)>,
 'Job13': <gurobi.Var yj[Job13] (value 1.0)>,
 'Job14': <gurobi.Var yj[Job14] (value 1.0)>,
 'Job15': <gurobi.Var yj[Job15] (value 1.0)>}

In [17]:

# start date
sj

{'Job1': <gurobi.Var sj[Job1] (value 1.0)>,
 'Job2': <gurobi.Var sj[Job2] (value 2.0)>,
 'Job3': <gurobi.Var sj[Job3] (value 3.0)>,
 'Job4': <gurobi.Var sj[Job4] (value 2.0)>,
 'Job5': <gurobi.Var sj[Job5] (value 4.0)>}

In [794]:
# end date
ej

{'Job1': <gurobi.Var ej[Job1] (value 2.0)>,
 'Job2': <gurobi.Var ej[Job2] (value 1.0)>,
 'Job3': <gurobi.Var ej[Job3] (value 4.0)>,
 'Job4': <gurobi.Var ej[Job4] (value 3.0)>,
 'Job5': <gurobi.Var ej[Job5] (value 5.0)>}

In [795]:
# length
nj

{'Job1': <gurobi.Var nj[Job1] (value 2.0)>,
 'Job2': <gurobi.Var nj[Job2] (value 2.0)>,
 'Job3': <gurobi.Var nj[Job3] (value 2.0)>,
 'Job4': <gurobi.Var nj[Job4] (value 2.0)>,
 'Job5': <gurobi.Var nj[Job5] (value 2.0)>}